In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent
# from art.estimators.classification import PyTorchClassifier

import time

In [2]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',

        }
head = pd.DataFrame([head])
head.to_csv("./RSLAD_100.csv", mode='a', index=False)


In [3]:
def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_test))
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    


        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        

        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,

        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("./RSLAD_100.csv", mode='a', index=False, header=False)



In [4]:
# def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
#     model.eval()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model.to(device)
    
#     all_preds = []
#     all_labels = []
#     probabilities = []

#     num_classes = len(np.unique(y_test))
    
#     X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
#     y_test_tensor = torch.tensor(y_test, dtype=torch.long)
#     test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
#     test_loader = DataLoader(dataset=test_dataset)

#     with torch.no_grad():
        
#         for inputs, labels in test_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             preds = torch.argmax(outputs, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#             probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
#         all_preds = np.array(all_preds)
#         all_labels = np.array(all_labels)
#         probabilities = np.array(probabilities)

#         np.save(f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense_Label/UNSW_Def10/y_pred_{attack_name}{eps}_Def10.npy", all_preds)


In [5]:
x_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
x_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_train.npy')
x_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_val.npy')
y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')
y_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_train.npy')
y_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_val.npy')

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [7]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [8]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.autograd import Variable

class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [10]:
# Initialize model, optimizer, and loss function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

# Early stopping variables
min_delta = 0.001
patience = 5
patience_counter = 0
best_loss = float('inf')


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import numpy as np

def attack_pgd(model,train_batch_data,train_batch_labels,attack_iters=10,step_size=2/255.0,epsilon=8.0/255.0):
    ce_loss = torch.nn.CrossEntropyLoss().cuda()
    train_ifgsm_data = train_batch_data.detach() + torch.zeros_like(train_batch_data).uniform_(-epsilon,epsilon)
    train_ifgsm_data = torch.clamp(train_ifgsm_data,0,1)
    for i in range(attack_iters):
        train_ifgsm_data.requires_grad_()
        logits = model(train_ifgsm_data)
        loss = ce_loss(logits,train_batch_labels.cuda())
        loss.backward()
        train_grad = train_ifgsm_data.grad.detach()
        train_ifgsm_data = train_ifgsm_data + step_size*torch.sign(train_grad)
        train_ifgsm_data = torch.clamp(train_ifgsm_data.detach(),0,1)
        train_ifgsm_pert = train_ifgsm_data - train_batch_data
        train_ifgsm_pert = torch.clamp(train_ifgsm_pert,-epsilon,epsilon)
        train_ifgsm_data = train_batch_data + train_ifgsm_pert
        train_ifgsm_data = train_ifgsm_data.detach()
    return train_ifgsm_data

def rslad_inner_loss(model,
                teacher_logits,
                x_natural,
                y,
                optimizer,
                step_size=0.003,
                epsilon=0.031,
                perturb_steps=10,
                beta=6.0):
    # define KL-loss
    criterion_kl = nn.KLDivLoss(size_average=False,reduce=False)
    model.eval()
    batch_size = len(x_natural)
    # generate adversarial example
    x_adv = x_natural.detach() + 0.001 * torch.randn(x_natural.shape).cuda().detach()

    for _ in range(perturb_steps):
        x_adv.requires_grad_()
        with torch.enable_grad():
            loss_kl = criterion_kl(F.log_softmax(model(x_adv), dim=1),
                                       F.softmax(teacher_logits, dim=1))
            loss_kl = torch.sum(loss_kl)
        grad = torch.autograd.grad(loss_kl, [x_adv])[0]
        x_adv = x_adv.detach() + step_size * torch.sign(grad.detach())
        x_adv = torch.min(torch.max(x_adv, x_natural - epsilon), x_natural + epsilon)
        x_adv = torch.clamp(x_adv, 0.0, 1.0)

    model.train()

    x_adv = Variable(torch.clamp(x_adv, 0.0, 1.0), requires_grad=False)
    # zero gradient
    optimizer.zero_grad()
    logits = model(x_adv)
    return logits
  

In [12]:
import os
import argparse
import torch
# from rslad_loss import *
# from cifar10_models import *
import torchvision
from torchvision import datasets, transforms
import time
# # we fix the random seed to 0, in the same computer, this method can make the results same as before.
# torch.manual_seed(0)
# torch.cuda.manual_seed_all(0)
# torch.backends.cudnn.deterministic = True

# prefix = 'mobilenet_v2-CIFAR10_RSLAD'
epochs = 10
batch_size = 128
epsilon = 8/255.0
multiply = 1



student = DNNModel(input_shape,output_shape )
student = torch.nn.DataParallel(student)
student = student.cuda()
student.train()
optimizer = optim.SGD(student.parameters(), lr=0.1, momentum=0.9, weight_decay=2e-4)
def kl_loss(a,b):
    loss = -a*b + torch.log(b+1e-5)*b
    return loss
teacher = DNNModel(input_shape,output_shape )
teacher.load_state_dict(torch.load('/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/dnn_pytorch.pt'))
teacher = torch.nn.DataParallel(teacher)
teacher = teacher.cuda()
teacher.eval()

/tmp/ipykernel_3644/291553794.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  teacher.load_state_dict(torch.load('/home/jovyan/Sample_Based_Extension/UNSW/transfer_atta

DataParallel(
  (module): DNNModel(
    (fc1): Linear(in_features=56, out_features=50, bias=True)
    (fc2): Linear(in_features=50, out_features=30, bias=True)
    (fc3): Linear(in_features=30, out_features=20, bias=True)
    (fc4): Linear(in_features=20, out_features=2, bias=True)
  )
)

In [13]:


# for epoch in range(0,epochs):
#     train_loss = 0.0
#     for step,(train_batch_data,train_batch_labels) in enumerate(train_loader):
#         student.train()
#         train_batch_data = train_batch_data.float().cuda()
#         train_batch_labels = train_batch_labels.cuda()
#         optimizer.zero_grad()
#         with torch.no_grad():
#             teacher_logits = teacher(train_batch_data)

#         adv_logits = rslad_inner_loss(student,teacher_logits,train_batch_data,train_batch_labels,optimizer,step_size=2/255.0,epsilon=epsilon,perturb_steps=3)
#         student.train()
#         nat_logits = student(train_batch_data)
#         kl_Loss1 = kl_loss(torch.log(F.softmax(adv_logits,dim=1)),F.softmax(teacher_logits.detach(),dim=1))
#         kl_Loss2 = kl_loss(torch.log(F.softmax(nat_logits,dim=1)),F.softmax(teacher_logits.detach(),dim=1))
#         # multiply 10 to keep consistent with CIFAR-10 dataset
#         kl_Loss1 = multiply*torch.mean(kl_Loss1)
#         kl_Loss2 = multiply*torch.mean(kl_Loss2)
#         loss = 5.0/6.0*kl_Loss1 + 1.0/6.0*kl_Loss2
#         loss.backward()
#         optimizer.step()
#         if step % 1000 == 0:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, step * len(train_batch_data), len(train_loader.dataset),
#                 100. * step / len(train_loader), loss.item()))
#         train_loss += loss.item()

#     avg_train_loss = train_loss / len(train_loader)

#     model.eval()
#     val_train_loss = 0.0
#     correct_predictions = 0
#     with torch.no_grad():
#         for inputs, labels in val_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
            
#             outputs = student(inputs)
#             kl_Loss1 = kl_loss(torch.log(F.softmax(adv_logits,dim=1)),F.softmax(teacher_logits.detach(),dim=1))
#             kl_Loss2 = kl_loss(torch.log(F.softmax(nat_logits,dim=1)),F.softmax(teacher_logits.detach(),dim=1))
#             # multiply 10 to keep consistent with CIFAR-10 dataset
#             kl_Loss1 = multiply*torch.mean(kl_Loss1)
#             kl_Loss2 = multiply*torch.mean(kl_Loss2)
#             loss = 5.0/6.0*kl_Loss1 + 1.0/6.0*kl_Loss2
            
#             val_train_loss += loss.item()
#             _, predicted = torch.max(outputs.data, 1)
#             correct_predictions += (predicted == labels).sum().item()

#     avg_val_loss = val_train_loss / len(val_loader)
#     val_accuracy = correct_predictions / len(val_dataset)

#     print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
#     # print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}")

    
#     # if (epoch%2 == 0):
#     #     test_accs = []
#     #     student.eval()
#     #     for step,(test_batch_data,test_batch_labels) in enumerate(test_loader):
#     #         test_ifgsm_data = attack_pgd(student,test_batch_data,test_batch_labels,attack_iters=20,step_size=0.003,epsilon=8.0/255.0)
#     #         logits = student(test_ifgsm_data)
#     #         predictions = np.argmax(logits.cpu().detach().numpy(),axis=1)
#     #         predictions = predictions - test_batch_labels.cpu().detach().numpy()
#     #         test_accs = test_accs + predictions.tolist()
#     #     test_accs = np.array(test_accs)
#     #     test_acc = np.sum(test_accs==0)/len(test_accs)
#     #     print('robust acc',np.sum(test_accs==0)/len(test_accs))
#     #     # torch.save(student.state_dict(),'./models/'+prefix+str(np.sum(test_accs==0)/len(test_accs))+'.pth')


#     # # Early stopping check using min_delta
#     # if best_loss - avg_val_loss > min_delta:
#     #     best_loss = avg_val_loss
#     #     patience_counter = 0
#     # else:
#     #     patience_counter += 1

#     # if patience_counter >= patience:
#     #     print("Early stopping triggered")
#     #     break

    

In [14]:
student.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/RSLAD/RSLAD_100.pt"))

/tmp/ipykernel_3644/2390732642.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/

<All keys matched successfully>

In [15]:
# calculate_performance_metrics(x_test, y_test, student, 'DNN', 'baseline', '0')

In [16]:
def print_empty_file():
    new_row = {
        "model" : "0",
        "attack_model" : "0",
        'epsilon': "0",
        'Accuracy': "0",
        'Macro Precision': "0",
        'Weighted Precision': "0",
        'Macro Recall': "0",
        'Weighted Recall': "0",
        'Macro F1': "0",
        'Weighted F1': "0",
    }
    new_row_df = pd.DataFrame([new_row])
    new_row_df.to_csv("./RSLAD_100.csv", mode='a', index=False, header=False)

In [17]:
# percentage = ["100", "50", "20", "1"]
# model_name = ["XGB", "RF", "DT" ]

percentage = ["10"]
model_name = ["RF"]

In [18]:
epsilon = 0
Def = "Def10"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}"
                calculate_performance_metrics(x_test_adv, y_test_adv, student, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start RF 10 baseline
(73, 56) (73,)
Accuracy: 0.8904109589041096

macro
Precision: 0.8879166666666667
Recall: 0.8747987117552335
F1 Score: 0.8805237315875614

weighted
Precision: 0.8899771689497717
Recall: 0.8904109589041096
F1 Score: 0.8894693182072954

start RF 10 BIM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input10/x_test_adv_BIM_Def10.npy not found
start RF 10 FGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input10/x_test_adv_FGSM_Def10.npy not found
start RF 10 PGD
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input10/x_test_adv_PGD_Def10.npy not found
start RF 10 DF
(265, 56) (265,)
Accuracy: 0.9584905660377359

macro
Precision: 0.49224806201550386
Recall: 0.48659003831417624
F1 Score: 0.48940269749518306

weighted
Precision: 0.9696358051777095
Recall: 0.9584905660377359
F1 Score: 0.9640309739339078

start RF 10 AutoPGD
(138, 56) (138,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9347826086956522

macro
Precision: 0.5
Recall: 0.4673913043478261
F1 Score: 0.48314606741573035

weighted
Precision: 1.0
Recall: 0.9347826086956522
F1 Score: 0.9662921348314607

start RF 10 ZOO
(196, 56) (196,)
Accuracy: 0.9540816326530612

macro
Precision: 0.9338235294117647
Recall: 0.9671532846715328
F1 Score: 0.9475857970583865

weighted
Precision: 0.9601590636254501
Recall: 0.9540816326530612
F1 Score: 0.9549289155567146

start RF 10 CaFA
(945, 56) (945,)
Accuracy: 0.7576719576719577

macro
Precision: 0.7380695164375551
Recall: 0.7326598413451759
F1 Score: 0.7350734346250041

weighted
Precision: 0.7553972716584092
Recall: 0.7576719576719577
F1 Score: 0.756280019658196

start RF 10 SINIFGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input10/x_test_adv_SINIFGSM_Def10.npy not found
start RF 10 VNIFGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input10/x_test_adv_VNIFGSM_Def10.npy not found


In [19]:
epsilon = 0
Def = "Def10"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ExcludeCaFA"
                calculate_performance_metrics(x_test_adv, y_test_adv, student, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start RF 10 baseline
(95, 56) (95,)
Accuracy: 0.8

macro
Precision: 0.6408268733850129
Recall: 0.5786435786435786
F1 Score: 0.5898659395591911

weighted
Precision: 0.7627906976744186
Recall: 0.8
F1 Score: 0.7721881390593046

start RF 10 BIM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ExcludeCaFA/UNSW_Input10/x_test_adv_BIM_Def10.npy not found
start RF 10 FGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ExcludeCaFA/UNSW_Input10/x_test_adv_FGSM_Def10.npy not found
start RF 10 PGD
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ExcludeCaFA/UNSW_Input10/x_test_adv_PGD_Def10.npy not found
start RF 10 DF
(247, 56) (247,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.97165991902834

macro
Precision: 0.5
Recall: 0.48582995951417
F1 Score: 0.4928131416837782

weighted
Precision: 1.0
Recall: 0.97165991902834
F1 Score: 0.9856262833675564

start RF 10 AutoPGD
(231, 56) (231,)
Accuracy: 0.935064935064935

macro
Precision: 0.6428571428571428
Recall: 0.9666666666666667
F1 Score: 0.7049808429118773

weighted
Precision: 0.9814471243042672
Recall: 0.935064935064935
F1 Score: 0.9519828830173658

start RF 10 ZOO
(253, 56) (253,)
Accuracy: 0.8695652173913043

macro
Precision: 0.6347826086956522
Recall: 0.6208065062690613
F1 Score: 0.6271602732996918

weighted
Precision: 0.8627599243856333
Recall: 0.8695652173913043
F1 Score: 0.8660004388017218

start RF 10 CaFA
(1, 56) (1,)
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start RF 10 SINIFGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ExcludeCaFA/UNSW_Input10/x_test_adv_SINIFGSM_Def10.npy not found
start RF

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

In [20]:
epsilon = 0
Def = "Def10"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

total_num_attack = ["2", "3"]
for num_attack in total_num_attack:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    print(x_test_adv.shape, y_test_adv.shape)
    
                    m_per_name = f"{m_name}{p}_Exclude{num_attack}Attack"
                    calculate_performance_metrics(x_test_adv, y_test_adv, student, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()


start RF 10 baseline
(83, 56) (83,)
Accuracy: 0.8192771084337349

macro
Precision: 0.6383561643835616
Recall: 0.610989010989011
F1 Score: 0.6214654910307085

weighted
Precision: 0.8020465423337183
Recall: 0.8192771084337349
F1 Score: 0.8093865275635838

start RF 10 BIM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude2Attack/UNSW_Input10/x_test_adv_BIM_Def10.npy not found
start RF 10 FGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude2Attack/UNSW_Input10/x_test_adv_FGSM_Def10.npy not found
start RF 10 PGD
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude2Attack/UNSW_Input10/x_test_adv_PGD_Def10.npy not found
start RF 10 DF
(236, 56) (236,)
Accuracy: 0.9576271186440678

macro
Precision: 0.6045031815956926
Recall: 0.6848484848484848
F1 Score: 0.6319401122894572

weighted
Precision: 0.970585795232999
Recall: 0.9576271186440678
F1 Score: 0.9634953529927994

start RF 10 AutoPGD
(186, 56) (186,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9032258064516129

macro
Precision: 0.5
Recall: 0.45161290322580644
F1 Score: 0.4745762711864407

weighted
Precision: 1.0
Recall: 0.9032258064516129
F1 Score: 0.9491525423728814

start RF 10 ZOO
(227, 56) (227,)
Accuracy: 0.8854625550660793

macro
Precision: 0.6466666666666666
Recall: 0.7218487394957983
F1 Score: 0.6728381374722838

weighted
Precision: 0.9130690161527165
Recall: 0.8854625550660793
F1 Score: 0.8970813756996201

start RF 10 CaFA
(2, 56) (2,)
Accuracy: 0.5

macro
Precision: 0.5
Recall: 0.25
F1 Score: 0.3333333333333333

weighted
Precision: 1.0
Recall: 0.5
F1 Score: 0.6666666666666666

start RF 10 SINIFGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude2Attack/UNSW_Input10/x_test_adv_SINIFGSM_Def10.npy not found
start RF 10 VNIFGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude2Attack/UNSW_Input10/x_test_adv_VNIFGSM_Def10.npy not found
start RF 10 baseline
(118, 56) (118,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8559322033898306

macro
Precision: 0.7720125786163522
Recall: 0.6698576337751596
F1 Score: 0.7005523212419764

weighted
Precision: 0.8398624880076752
Recall: 0.8559322033898306
F1 Score: 0.8394802158682931

start RF 10 BIM
(13, 56) (13,)
Accuracy: 0.9230769230769231

macro
Precision: 0.5
Recall: 0.46153846153846156
F1 Score: 0.48

weighted
Precision: 1.0
Recall: 0.9230769230769231
F1 Score: 0.9600000000000001

start RF 10 FGSM
(13, 56) (13,)
Accuracy: 0.8461538461538461

macro
Precision: 0.4583333333333333
Recall: 0.4583333333333333
F1 Score: 0.4583333333333333

weighted
Precision: 0.8461538461538461
Recall: 0.8461538461538461
F1 Score: 0.8461538461538461

start RF 10 PGD
(13, 56) (13,)
Accuracy: 0.9230769230769231

macro
Precision: 0.5
Recall: 0.46153846153846156
F1 Score: 0.48

weighted
Precision: 1.0
Recall: 0.9230769230769231
F1 Score: 0.9600000000000001

start RF 10 DF
(25, 56) (25,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.p

Accuracy: 0.76

macro
Precision: 0.38
Recall: 0.5
F1 Score: 0.4318181818181818

weighted
Precision: 0.5776
Recall: 0.76
F1 Score: 0.6563636363636364

start RF 10 AutoPGD
(271, 56) (271,)
Accuracy: 0.915129151291513

macro
Precision: 0.644927536231884
Recall: 0.6528485409911996
F1 Score: 0.6487461256692026

weighted
Precision: 0.9174287395047863
Recall: 0.915129151291513
F1 Score: 0.9162578929255057

start RF 10 ZOO
(308, 56) (308,)
Accuracy: 0.9025974025974026

macro
Precision: 0.7457493080268881
Recall: 0.7054545454545454
F1 Score: 0.7230215827338129

weighted
Precision: 0.8951872563972207
Recall: 0.9025974025974026
F1 Score: 0.8982528263103803

start RF 10 CaFA
(4, 56) (4,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 SINIFGSM
(12, 56) (12,)
Accuracy: 0.9166666666666666

macro
Precision: 0.5
Recall: 0.4583333333333333
F1 Score: 0.4782608695652174

weighted
Precision: 1.0
Recall: 0.9166666666666666
F1 Sco

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [17]:
epsilon = 0

Def = "Def10"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

rec_list = ["Euclidean", "cosine", "manhattan"]
for rec in rec_list:
    for attack in attack_names:
        print(f"start {attack}")
        
        x_path = f"{base_path}/Recommendation_{rec}/x_test_adv_{attack}_{Def}.npy"
        y_path = f"{base_path}/Recommendation_{rec}/y_test_adv_{attack}_{Def}.npy"
    
        try:
            x_test_adv = np.load(x_path)
            y_test_adv = np.load(y_path)
            m_per_name = f"Recommendation_{rec}"
            calculate_performance_metrics(x_test_adv, y_test_adv, student, m_per_name, attack, epsilon)
        except FileNotFoundError:
            print(x_path, "not found")
            print_empty_file()



start baseline
Accuracy: 0.9651567944250871

macro
Precision: 0.9681528662420382
Recall: 0.9642857142857143
F1 Score: 0.9650341130604287

weighted
Precision: 0.9673761068820881
Recall: 0.9651567944250871
F1 Score: 0.9650846289164645

start BIM
Accuracy: 0.4702549575070821

macro
Precision: 0.5419140754982068
Recall: 0.5914602662363856
F1 Score: 0.42536114483032117

weighted
Precision: 0.8267500671111697
Recall: 0.4702549575070821
F1 Score: 0.5453307591118006

start FGSM
Accuracy: 0.4624145785876993

macro
Precision: 0.5344807219807219
Recall: 0.5357107903107414
F1 Score: 0.46218853820598005

weighted
Precision: 0.6150252351846884
Recall: 0.4624145785876993
F1 Score: 0.4664833054586458

start PGD
Accuracy: 0.4702549575070821

macro
Precision: 0.5419140754982068
Recall: 0.5914602662363856
F1 Score: 0.42536114483032117

weighted
Precision: 0.8267500671111697
Recall: 0.4702549575070821
F1 Score: 0.5453307591118006

start DF
Accuracy: 0.8825757575757576

macro
Precision: 0.7612927923001203


In [21]:
epsilon = 0
Def = "Def10"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_Cluster/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_Cluster/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_Cluster"
                calculate_performance_metrics(x_test_adv, y_test_adv, student, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start RF 10 baseline
(10283, 56) (10283,)
Accuracy: 0.9829816201497618

macro
Precision: 0.9856998941873527
Recall: 0.9793778376297069
F1 Score: 0.9822671942833106

weighted
Precision: 0.9833750063250895
Recall: 0.9829816201497618
F1 Score: 0.9829258921630861

start RF 10 BIM
(115, 56) (115,)
Accuracy: 0.9826086956521739

macro
Precision: 0.9806005398110661
Recall: 0.9806005398110661
F1 Score: 0.9806005398110661

weighted
Precision: 0.9826086956521739
Recall: 0.9826086956521739
F1 Score: 0.9826086956521739

start RF 10 FGSM
(3662, 56) (3662,)
Accuracy: 0.049153468050245765

macro
Precision: 0.5056404885470727
Recall: 0.5081540570671006
F1 Score: 0.04907149539178205

weighted
Precision: 0.9711353782817941
Recall: 0.049153468050245765
F1 Score: 0.05754361074006143

start RF 10 PGD
(115, 56) (115,)
Accuracy: 0.9826086956521739

macro
Precision: 0.9806005398110661
Recall: 0.9806005398110661
F1 Score: 0.9806005398110661

weighted
Precision: 0.9826086956521739
Recall: 0.9826086956521739
F1 S

In [22]:
epsilon = 0
Def = "Def10"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ActiveLearning/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ActiveLearning/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ActiveLearning"
                calculate_performance_metrics(x_test_adv, y_test_adv, student, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start RF 10 baseline
(46, 56) (46,)
Accuracy: 0.9782608695652174

macro
Precision: 0.9864864864864865
Recall: 0.95
F1 Score: 0.9668348954578225

weighted
Precision: 0.9788484136310224
Recall: 0.9782608695652174
F1 Score: 0.9778376853390176

start RF 10 BIM
(6, 56) (6,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 FGSM
(6, 56) (6,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 PGD
(6, 56) (6,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 DF
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ActiveLearning/UNSW_Input10/x_test_adv_DF_Def10.npy not found
start RF 10 AutoPGD
(93, 56) (93,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 ZOO
(133, 56) (133

In [23]:
epsilon = 0

Def = "Def10"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]


active_learning_name = ["DensityWeighted", "BatchMode"]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for ac_name in active_learning_name:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_ActiveLearning_{ac_name}/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_ActiveLearning_{ac_name}/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    m_per_name = f"{m_name}{p}_ActiveLearning_{ac_name}"
                    calculate_performance_metrics(x_test_adv, y_test_adv, student, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()



start RF 10 baseline
Accuracy: 0.8857408107829317

macro
Precision: 0.8834811225800516
Recall: 0.8878755031367184
F1 Score: 0.884797279950202

weighted
Precision: 0.8881170857974385
Recall: 0.8857408107829317
F1 Score: 0.88606595609649

start RF 10 BIM
Accuracy: 0.827727935652097

macro
Precision: 0.8340603783856892
Recall: 0.8208809529149791
F1 Score: 0.8237459619661409

weighted
Precision: 0.8315572774323288
Recall: 0.827727935652097
F1 Score: 0.8260021405291628

start RF 10 FGSM
Accuracy: 0.8330318139699069

macro
Precision: 0.8389872945456267
Recall: 0.8260179493165987
F1 Score: 0.8290683248552356

weighted
Precision: 0.8364902299982501
Recall: 0.8330318139699069
F1 Score: 0.8314101255810586

start RF 10 PGD
Accuracy: 0.827727935652097

macro
Precision: 0.8340603783856892
Recall: 0.8208809529149791
F1 Score: 0.8237459619661409

weighted
Precision: 0.8315572774323288
Recall: 0.827727935652097
F1 Score: 0.8260021405291628

start RF 10 DF
Accuracy: 0.6828561072852483

macro
Precision:

In [15]:

# torch.save(student.state_dict(), "./RSLAD_100.pt")

In [17]:
# import time

# epsilon_values = [0.01, 0.1, 0.2, 0.3]

# start_time = time.time()

# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, student, 'DNN', 'BIM', epsilon)

# end_time = time.time()
# result = end_time - start_time
# print(f"Execution Time: {result:.6f} seconds")